<a href="https://colab.research.google.com/github/misko/spf/blob/main/02_beamformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import namedtuple
from spf.rf import beamformer, Detector, speed_of_light, IQSource
import matplotlib.pyplot as plt
import numpy as np

"""

Given some guess of the source of direction we can shift the carrier frequency
phase of received samples at the N different receivers. If the guess of the
source direction is correct, the signal from the N different receivers should
interfer constructively after phase shift.

"""


def plot_space(ax, d):
    receiver_pos = np.vstack([pos for pos in d.receiver_positions])
    _max = receiver_pos.max()
    _min = receiver_pos.min()
    buffer = max(3, int((_max - _min) * 0.1))
    _max += buffer
    _min -= buffer

    center_mass = receiver_pos.mean(axis=0)

    source_vectors = [
        (source.pos - center_mass) / np.linalg.norm(source.pos - center_mass)
        for source in d.sources
    ]

    ax.set_xlim([_min, _max])
    ax.set_ylim([_min, _max])

    ax.scatter(receiver_pos[:, 0], receiver_pos[:, 1], label="Receivers")
    for source_vector in source_vectors:
        ax.quiver(
            center_mass[0],
            center_mass[1],
            source_vector[0][0],
            source_vector[0][1],
            scale=5,
            alpha=0.5,
            color="red",
            label="Source",
        )
    ax.legend()


Receiver = namedtuple("Receiver", ["pos"])

n = 15
fig, axs = plt.subplots(n, 4, figsize=(4 * 4, 4 * n))

for theta_idx, theta in enumerate(np.linspace(-np.pi / 2, np.pi / 2, n)):
    d = Detector(1e6)  # 1Mhz sampling
    sin_source_frequency = 12e3  # 10khz signal
    wavelength = speed_of_light / sin_source_frequency

    # Add a signal source very far away so that its farfield
    # and that assumptions hold. For nearby sources this will
    # not be true
    d.add_source(
        IQSource(
            [
                [
                    wavelength * 100000 * np.sin(theta),
                    wavelength * 100000 * np.cos(theta),
                ]
            ],
            sin_source_frequency,
            0,
        )
    )

    # the X,Y coordinates of our recievers
    d.add_receiver(Receiver([wavelength / 4, 0]))
    d.add_receiver(Receiver([-wavelength / 4, 0]))

    # get a simulated signal matrix. The math here is different
    # then that of what will happen on the SDR but the result I
    # think is similar
    # also the signal source is very far away so increase the
    # magnitude (gain?)
    signal_matrix = d.get_signal_matrix(0, 200 / 1e6) * 1e18

    n_receivers = d.n_receivers()

    # The beamformer will create 65 uniformily spaced directions
    # (approx 5deg apart) around the center of the receiver array.
    # For each direction it will compute the expected phase offset
    # between antenna 1 and antenna 2
    # It will then apply a correction to fix this offset and compute
    # the sum of the corrected vectors, and return their magnitude

    # For example, imagine that the distance between recievers is
    # half a wavelength

    # If a signal source is directly to the right of the array
    # then antenna 0 (right) will be exactly pi phase ahead
    # of antenna 1. If we take their sum it will be 0
    # cos(x) + cos(x+pi) = 0

    # If a signal source is directly to infront of the array
    # then antenna 0 (right) will be exactly 0 phase ahead
    # of antenna 1. If we take their sum it will be 0
    # cos(x) + cos(x) = 2cos(x)

    # given the direction of the source, we can compute the phase
    # offset , apply it, take the sum , and return the resulting
    # power of signal in that direction

    # This is what the beamformer does

    beam_thetas, beam_sds, _ = beamformer(
        d.all_receiver_pos(), signal_matrix, sin_source_frequency, spacing=64 + 1
    )

    for idx in range(signal_matrix.shape[0]):
        axs[theta_idx][idx].plot(
            signal_matrix[idx].real + signal_matrix[idx].imag, label="baseband"
        )
        axs[theta_idx][idx].plot(signal_matrix[idx].real, label="real I")
        axs[theta_idx][idx].plot(signal_matrix[idx].imag, label="imag Q")
        axs[theta_idx][idx].legend()
        axs[theta_idx][idx].set_title("Receiver %d" % idx)
        axs[theta_idx][idx].set_xlabel("Sample")

    beam_degrees = 360 * beam_thetas / (np.pi * 2)
    axs[theta_idx][n_receivers].plot(beam_degrees, beam_sds, alpha=0.1)
    axs[theta_idx][n_receivers].scatter(beam_degrees, beam_sds, s=0.5)
    axs[theta_idx][n_receivers].axvline(
        x=360 * theta / (2 * np.pi), label="Truth", color="red"
    )
    axs[theta_idx][n_receivers].set_title("Response")
    axs[theta_idx][n_receivers].set_xlabel("Theta")
    axs[theta_idx][n_receivers].legend()

    plot_space(axs[theta_idx][n_receivers + 1], d)


fig.tight_layout()

In [ ]:
beam_thetas

In [ ]:
d.all_receiver_pos()

In [ ]:
d.all_receiver_pos()

In [ ]:
d.source_positions

In [ ]:
wavelength